In [1]:
import cv2
import numpy as np
from tqdm import tqdm

Open the video file, get the frame dimensions and frame rate

In [14]:
cap = cv2.VideoCapture("CellA_noDoxycycline.avi")

# Get properties of the video
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(length)

356953


In [4]:
processed_movie = []
ret = True
while ret:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Convert frame to grayscale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Crop the frame 20% from left and right
    crop_width = int(frame_width * 0.2)
    cropped_frame = gray_frame[:, crop_width:-crop_width]

    # Apply thresholding
    mask = cropped_frame > 150 # threshold seems to be good, but could improve
    mask = mask.astype(np.uint8)
    
    if np.sum(mask) > 20:
        processed_movie.append(frame)
    
cv2.destroyAllWindows()

In [10]:
cap.release()
processed_movie = np.array(processed_movie)
processed_movie.shape


(21438, 256, 128, 3)

In [6]:
# play the video (press q to quit at any time)
def play_video(vid):
    for idx in range(vid.shape[0]):
        cv2.imshow('video', np.array(vid[idx,...], dtype=np.uint8))
        if cv2.waitKey(100) & 0xFF == ord('q'):
            break

    cv2.destroyAllWindows()

In [7]:
play_video(processed_movie)

In [13]:
out = cv2.VideoWriter('output.avi', cv2.VideoWriter_fourcc(*"MJPG"), fps, (frame_width,frame_height))

for i in processed_movie:

    out.write(i)

out.release()
